In [14]:
import torch
import functools
import numpy  as np
from IPython.core.debugger import set_trace

https://github.com/karpathy/micrograd

In [55]:
# make base class
class Tensor:
    def __init__(self, data):
        if type(data) != np.ndarray:
            print('error type data %r' % data)
            assert(False)
        self.data = data        
        self.grad = None
        self._ctx = None
    
    def __repr__(self):
        # tensor([1.]))
        return f'tensor({self.data})'

    def backward(self, fill = True):
        
        if self._ctx is None:
            return

        if self.grad is None and fill:
            # iniy first grad ones
            assert self.data.size == 1
            self.grad = np.ones_like(self.data)
        
        assert(self.grad is not None)  

        """
        _ctx.backward return from <class '__main__.Mul'> 
        and method Mul.backward return y * grad_out, x * grad_out
        
        """      
        grads = self._ctx.backward(self._ctx, self.grad)
        if len(self._ctx.parents) == 1:
            grads = [grads]
       
        for t, g in zip(self._ctx.parents, grads):
            # t  Tensor
            if g.shape != t.data.shape:
                print('grad shape must match tensor shape')
                assert(False)           
            # add grad to Tensor
            t.grad = g          
            # t.backward(False)
         

"""
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html#:~:text=beginner%2Fexamples_autograd%2Ftwo_layer_net_custom_function-,PyTorch%3A%20Defining%20New%20autograd%20Functions,PyTorch%20autograd%20to%20compute%20gradients.
В прямом проходе мы получаем тензор, содержащий ввод и возврат
Тензор, содержащий вывод. 

ctx - это объект контекста, который можно использовать
хранить информацию для обратных вычислений. Вы можете кешировать произвольные
объекты для использования в обратном проходе с помощью метода ctx.save_for_backward.


При обратном проходе мы получаем тензор, содержащий градиент потери
относительно выхода, и нам нужно вычислить градиент потерь относительно входа.

"""

class Function:

    def __init__(self, *tensor):
        self.parents = tensor
        self.saved_tensors = []
    
    def save_for_backward(self, *x):
        self.saved_tensors.extend(x)    

    def apply(self, arg, *x):
        """
        почему мы здесь а потому что fn.apply и передаем аргумент fn
        a = Tensor(np.array([1]))
        b = Tensor(np.array([3]))
        
        a.mul(b)
        
        arg: <class '__main__.Mul'> , fn
        self.data -> Tensor.data->[1] a
        [3] = b        
        
        """
     
        ctx = arg(self, *x)         
        ret = Tensor(arg.forward(ctx, self.data, *[t.data for t in x]))
        ret._ctx = ctx
        return ret


def register(name, fn):
    """
    class A:
        print('hell')
    a = A()
    setattr(a, 'oo', lambda x: x *2)
    a.oo(2)
    >4

    we add mul to class Tensor    

    partialmethod(fumc, arg)
    """    
    setattr(Tensor, name, functools.partialmethod(fn.apply, fn))


class Mul(Function):
    """
    out = x.mul.y
    back
    out/dx, out/dy
    
    """

    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x * y

    @staticmethod
    def backward(ctx, grad_out):       
        # set_trace()
        x, y = ctx.saved_tensors       
        return y * grad_out, x * grad_out

register('mul', Mul)

In [ ]:
#  copy paste
# balnck 
# class (Function):

#     @staticmethod
#     def forward(ctx, x, y):
#         ctx.save_for_backward(x, y)
#         return 

#     @staticmethod
#     def backward(ctx, grad_out):     
#         x, y = ctx.saved_tensors       
#         return 

In [69]:
a = Tensor(np.array([1]))
b = Tensor(np.array([3]))
c = a.mul(b)


In [70]:
c

tensor([3])

In [71]:
a.grad, b.grad

(None, None)

In [72]:
c.backward()

In [73]:
a.grad, b.grad

(array([3]), array([1]))

In [74]:
a  = torch.tensor([1.], requires_grad=True)
b  = torch.tensor([3.], requires_grad=True)
c = a.matmul(b)
c.backward()

In [75]:
c

tensor(3., grad_fn=<DotBackward>)

In [76]:
a.grad, b.grad

(tensor([3.]), tensor([1.]))

In [ ]:
class Add(Function):
    """sum"""
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x + y

    @staticmethod
    def backward(ctx, grad_out):     
        x, y = ctx.saved_tensors       
        return grad_out, grad_out


class ReLU(Function):
    """relu"""
    @staticmethod
    def forward(ctx, in_val):
        ctx.save_for_backward(in_val)
        return np.maximum(in_val, 0)

    @staticmethod
    def backward(ctx, grad_out):     
        in_val = ctx.saved_tensors   
        grad_out[in_val < 0] = 0   
        return grad_out

class Sum(Function):
    """sum"""
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x + y

    @staticmethod
    def backward(ctx, grad_out):     
        x, y = ctx.saved_tensors       
        return grad_out, grad_out

https://docs.sympy.org/latest/tutorial/calculus.html

In [77]:
from sympy import *

In [82]:
x, y = symbols('x, y')
diff(x+y, x)

1